In [1]:
import os
import sys

os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.6 (default, Oct 26 2016 20:30:19)
SparkSession available as 'spark'.


In [2]:
import json
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

In [3]:
my_user_id = 592

In [5]:
t = sc.textFile('ml-100k/u.data')
t.take(3)
t2=t.map(lambda x: x.split())
t2=t2.map(lambda x: (int(x[0]), int(x[1]), int(x[2])))
t2.cache()

PythonRDD[5] at RDD at PythonRDD.scala:48

user id | item id | rating

In [6]:
t2.take(5)

[(196, 242, 3), (186, 302, 3), (22, 377, 1), (244, 51, 2), (166, 346, 1)]

# Часть1. Найдем основные характеристики данных movielens:

1. Найдите количество всех пользователей и количество всех фильмов в данных (общее число пользователей и фильмов в датасете).

In [65]:
users_count = len(t2.map(lambda x: x[0]).countByValue())
users_count

943

In [66]:
films_count = len(t2.map(lambda x: x[1]).countByValue())
films_count

1682

Сколько пользователь в среднем ставит рейтингов? (количество рейтингов/количество пользователей) – average_user_ratings

In [67]:
average_user_ratings = float(t2.count())/users_count
average_user_ratings

106.04453870625663

Сколько фильм в среднем имеет рейтингов (количество рейтингов/количество фильмов)? – average_film_ratings

In [68]:
average_film_ratings = float(t2.count())/films_count
average_film_ratings

59.45303210463734

Найдите процент заполненных ячеек в данных (количество рейтингов/(количество пользователей*количество фильмов)) – completeness

In [18]:
completeness = float(t2.count())/(films_count*users_count)
completeness

0.06304669364224531

Какой тип CF вы бы использовали на данном датасете? Ответ не оценивается:
user-user
item-item

In [19]:
# user-user

# Часть 2. User-user CF:

Для каждого пользователя найдите его средний рейтинг (сумма рейтингов пользователя/количество рейтингов пользователя). Здесь Ia - множество фильмов, по которым у пользователя есть рейтинги rui. Здесь и далее |Ia| обозначает количество элементов в множестве Ia. 

In [7]:
df = spark.createDataFrame(t2, ["user_id", "item_id", "rating"])


In [8]:
df.take(5)

[Row(user_id=196, item_id=242, rating=3),
 Row(user_id=186, item_id=302, rating=3),
 Row(user_id=22, item_id=377, rating=1),
 Row(user_id=244, item_id=51, rating=2),
 Row(user_id=166, item_id=346, rating=1)]

In [9]:
df_avg_rating_by_user = df.groupBy("user_id").avg("rating").withColumnRenamed("avg(rating)","avg_user_rating")
df_avg_rating_by_user.take(5)

[Row(user_id=26, avg_user_rating=2.94392523364486),
 Row(user_id=29, avg_user_rating=3.6470588235294117),
 Row(user_id=474, avg_user_rating=4.08256880733945),
 Row(user_id=191, avg_user_rating=3.6296296296296298),
 Row(user_id=65, avg_user_rating=3.9375)]

Для выданного вам пользователя (ID в личном кабинете - my_user_id):


Посчитайте меру близости Пирсона выданного вам пользователя со всеми остальными пользователями. Обратите внимание, что корреляция Пирсона считается только на пересечении, то есть вклад вносят только фильмы, оцененные совместно (Ia,Iu - множества оцененных пользователями a и u фильмов). Корреляция с константой (ситуация, когда у пользователя все оценки одинаковые) равна нулю.
Формат: ID пользователя; корреляция Пирсона.


In [23]:
#avg_rating_by_user = df_avg_rating_by_user.collect()

In [24]:
#avg_rating_by_user[1][0]

In [10]:
my_user_average = df_avg_rating_by_user.filter(df.user_id==my_user_id).take(1)[0][1]

In [11]:
df.filter(df.user_id==my_user_id).registerTempTable("my_user_films")
df.registerTempTable("u_data")
df_avg_rating_by_user.registerTempTable("avg_rating_by_user")

Склеим фильмы которые оценивал наш юзер с фильмами других юзеров

In [12]:
pearson_prepare = spark.sql("SELECT  \
        u_data.user_id as user_id, \
        u_data.item_id as item_id,\
        u_data.rating as rating,  \
        my_user_films.rating as my_user_rating \
    FROM \
        u_data \
    INNER JOIN \
        my_user_films \
    ON \
        u_data.item_id = my_user_films.item_id \
    ORDER BY \
        u_data.user_id ")

In [13]:
#spark.sql("SHOW TABLES").show(3)
pearson_prepare.show(10)

+-------+-------+------+--------------+
|user_id|item_id|rating|my_user_rating|
+-------+-------+------+--------------+
|      1|    188|     3|             5|
|      1|      7|     4|             5|
|      1|    184|     4|             5|
|      1|    198|     5|             5|
|      1|    196|     5|             5|
|      1|    243|     1|             1|
|      1|    237|     2|             4|
|      1|    191|     5|             5|
|      1|     22|     4|             5|
|      1|    222|     4|             1|
+-------+-------+------+--------------+
only showing top 10 rows



In [14]:
pearson_prepare.registerTempTable("pearson_prepare")
pearson_prepare.show(3)

+-------+-------+------+--------------+
|user_id|item_id|rating|my_user_rating|
+-------+-------+------+--------------+
|      1|    243|     1|             1|
|      1|    222|     4|             1|
|      1|    191|     5|             5|
+-------+-------+------+--------------+
only showing top 3 rows



In [15]:
pearson_prepare2 = spark.sql("SELECT  \
        pearson_prepare.user_id as user_id, \
        pearson_prepare.item_id as item_id,\
        pearson_prepare.rating as rating,  \
        pearson_prepare.my_user_rating  as my_user_rating, \
        avg_rating_by_user.avg_user_rating as avg_user_rating \
    FROM \
        pearson_prepare \
    INNER JOIN \
        avg_rating_by_user \
    ON \
        pearson_prepare.user_id = avg_rating_by_user.user_id \
    ORDER BY \
        pearson_prepare.user_id")

In [16]:
my_pearson = pearson_prepare2.collect()

In [44]:
my_pearson

[Row(user_id=1, item_id=191, rating=5, my_user_rating=5, avg_user_rating=3.610294117647059),
 Row(user_id=1, item_id=222, rating=4, my_user_rating=1, avg_user_rating=3.610294117647059),
 Row(user_id=1, item_id=243, rating=1, my_user_rating=1, avg_user_rating=3.610294117647059),
 Row(user_id=1, item_id=237, rating=2, my_user_rating=4, avg_user_rating=3.610294117647059),
 Row(user_id=1, item_id=22, rating=4, my_user_rating=5, avg_user_rating=3.610294117647059),
 Row(user_id=1, item_id=198, rating=5, my_user_rating=5, avg_user_rating=3.610294117647059),
 Row(user_id=1, item_id=196, rating=5, my_user_rating=5, avg_user_rating=3.610294117647059),
 Row(user_id=1, item_id=7, rating=4, my_user_rating=5, avg_user_rating=3.610294117647059),
 Row(user_id=1, item_id=184, rating=4, my_user_rating=5, avg_user_rating=3.610294117647059),
 Row(user_id=1, item_id=188, rating=3, my_user_rating=5, avg_user_rating=3.610294117647059),
 Row(user_id=1, item_id=202, rating=5, my_user_rating=5, avg_user_rating=

In [45]:
from itertools import imap

def pearson(x, y):
    # Assume len(x) == len(y)
    sum_x = float(sum(x))
    sum_y = float(sum(y))
    sum_x_sq = sum(map(lambda x: pow(x, 2), x))
    print 'sq1',sum_x_sq
    sum_y_sq = sum(map(lambda x: pow(x, 2), y))
    print 'sq2',sum_y_sq
    num = sum(imap(lambda x, y: x * y, x, y))
    den = pow((sum_x_sq ),0.5) * pow((sum_y_sq), 0.5)
    if den == 0: return 0
    return num / den

In [47]:
x = []
y = []
pears = {}
prev = None
for each in sorted(my_pearson):
    if each[0] != prev and prev != None:
        print each[0]
        pears[prev]=pearson(x, y)
        x = []
        y = []
        x.append(float(each[2]-each[4]))
        y.append(float(each[3]-my_user_average))
    else:
        x.append(float(each[2]-each[4]))
        y.append(float(each[3]-my_user_average))
    prev = each[0]

2
sq1 196.252811419
sq2 179.461018519
3
sq1 37.9719042664
sq2 60.3979089506
4
sq1 56.0716735254
sq2 80.4867978395
5
sq1 13.1111111111
sq2 34.9486419753
6
sq1 110.903510204
sq2 89.3939429012
7
sq1 131.484086162
sq2 156.675756173
8
sq1 154.484228091
sq2 185.366234568
9
sq1 60.3590922149
sq2 101.511967593
10
sq1 4.78512396694
sq2 16.172507716
11
sq1 34.7257797732
sq2 109.507214506
12
sq1 79.9596166173
sq2 98.1807947531
13
sq1 11.4805843906
sq2 41.284529321
14
sq1 476.676575294
sq2 423.521805556
15
sq1 80.4497084548
sq2 70.2099305556
16
sq1 123.296875
sq2 140.176257716
17
sq1 49.3287755102
sq2 73.1141280864
18
sq1 34.0331632653
sq2 55.4116820988
19
sq1 89.1840242933
sq2 121.130347222
20
sq1 9.635
sq2 16.7182561728
21
sq1 47.1605902778
sq2 57.6887808642
22
sq1 156.77016947
sq2 223.229490741
23
sq1 104.030578613
sq2 76.8142361111
24
sq1 80.5685276962
sq2 108.262145062
25
sq1 35.7958477509
sq2 59.7122530864
26
sq1 20.6081525312
sq2 54.1766512346
27
sq1 53.8154423967
sq2 115.594513889
28
sq1 1

In [35]:
pears

{1: 0.3865583725768982,
 2: 0.30472178240875114,
 3: -0.371374053121275,
 4: 0.2054476567481995,
 5: 0.49747952276826707,
 6: 0.28256394823809566,
 7: 0.24484982974106576,
 8: 0.4182653845472005,
 9: 0.31901567835660805,
 10: 0.2406383191133872,
 11: 0.2590659606297791,
 12: -0.28283310688396657,
 13: 0.3095756494103931,
 14: 0.07813638143950367,
 15: 0.17714544401945886,
 16: 0.2724351100118296,
 17: 0.3911349925311021,
 18: 0.18984283032262594,
 19: -0.13998222666266416,
 20: -0.206011329377457,
 21: 0.3486242147681669,
 22: 0.6226608889523221,
 23: 0.2681026842458305,
 24: 0.33501185788582455,
 25: 0.2634918216097433,
 26: 0.5082999683305173,
 27: -0.009531892804397256,
 28: 0.4517470347573546,
 29: 0.6632094114848543,
 30: 0.1939555496519415,
 31: 0.24841446096219025,
 32: 0.45324243146539767,
 33: -0.09811172390904148,
 34: 0.044099109684917176,
 35: 0.27687860809315856,
 36: -0.6335976273011583,
 37: 0.41194770631948935,
 38: 0.1361265645875039,
 39: 0.7853273953257475,
 40: -0.0

Посчитайте поправочный коэффициент для корреляции Пирсона на нехватку данных

In [20]:
from itertools import imap

def pearson_corrections(x, y):
    # Assume len(x) == len(y)
    n = len(x)
    if n >= 50:
        correction = 1
    else: 
        correction = float(n) / 50
    sum_x = float(sum(x))
    sum_y = float(sum(y))
    sum_x_sq = sum(map(lambda x: pow(x, 2), x))
    sum_y_sq = sum(map(lambda x: pow(x, 2), y))
    num = sum(imap(lambda x, y: x * y, x, y))
    den = pow((sum_x_sq ),0.5) * pow((sum_y_sq), 0.5)
    if den == 0: return 0
    return correction * num / den

In [21]:
x = []
y = []
pears_correct = {}
prev = None
for each in sorted(my_pearson):
    if each[0] != prev and prev != None:
        pears_correct[prev]=pearson_corrections(x, y)
        x = []
        y = []
        x.append(float(each[2]-each[4]))
        y.append(float(each[3]-my_user_average))
    else:
        x.append(float(each[2]-each[4]))
        y.append(float(each[3]-my_user_average))
    prev = each[0]

In [22]:
pears_correct

{1: 0.5557699266295894,
 2: 0.22980752390439801,
 3: 0.15771652842856923,
 4: 0.18066265432084594,
 5: 0.35408754148186355,
 6: 0.2519154360122085,
 7: 0.41127309894969005,
 8: 0.45183294407989816,
 9: 0.04011634433821513,
 10: 0.27940123331407396,
 11: 0.22036940273009234,
 12: 0.17782781275699477,
 13: 0.31818923039391817,
 14: -0.042866077539172424,
 15: 0.059037786335585214,
 16: 0.4634299096415456,
 17: 0.3229483749344469,
 18: 0.2560031842798742,
 19: -0.05772168230518562,
 20: -0.1702142530244231,
 21: 0.5644201213644926,
 22: 0.48905536576274916,
 23: 0.3352832800109451,
 24: 0.2782889312757537,
 25: 0.21262215698580056,
 26: 0.36756882432874777,
 27: 0.13396528726880047,
 28: 0.3871244044682399,
 29: 0.34265852627833,
 30: 0.2740833879655162,
 31: 0.12938886070763464,
 32: 0.3280756580253404,
 33: 0.06799404087027627,
 34: -0.014229587719315907,
 35: 0.0969013056033464,
 36: -0.14357213539196004,
 37: 0.35084942311851797,
 38: -0.022920962415506604,
 39: 0.017004158659886325,


Найдите 30 ближайших пользователей-соседей данного пользователя (pearson_neighbours), используя поправленную корреляцию Пирсона: 

In [23]:
pearson_neighbours = sorted(dict(pears_correct).items(), key=lambda x: x[1], reverse=True)[1:31]

Дальше мы будем использовать только этих соседей для прогноза. Заметим, что строя предсказание для некоторого фильма, мы из фиксированных 30 ближайших пользователей-соседей выбираем только тех, у кого есть оценки для данного фильма, то есть используемых пользователей-соседей для прогноза может быть сильно меньше. Можно находить k ближайших пользователей-соседей, у которых есть оценки по интересующему нас фильму, но это вычислительно гораздо сложнее, поэтому целесообразность этого следует проверять на практике (в рамках данной лабораторной, это лишнее усложнение). 


Для всех фильмов найдите прогноз оценки по формуле ниже. Здесь N(a)- множество пользователей-соседей, s(a,u)- мера близости пользователей из предыдущих пунктов, |s(a,u)|  - модуль меры близости. 

In [25]:
[int(row[0]) for row in pearson_neighbours]

[397,
 615,
 453,
 303,
 267,
 526,
 916,
 457,
 345,
 890,
 595,
 293,
 823,
 394,
 21,
 537,
 640,
 69,
 1,
 276,
 379,
 249,
 94,
 59,
 683,
 854,
 216,
 831,
 323,
 933]

ERROR! Session/line number was not unique in database. History logging moved to new session 189


Соберем все фильмы, за которые наш пользователь не голосовал

In [26]:
df_pearson_prepare4 = spark.sql("SELECT  \
        u_data.user_id as user_id, \
        u_data.item_id as item_id,\
        u_data.rating as rating,  \
        my_user_films.rating as my_user_rating \
    FROM \
        u_data \
    LEFT OUTER JOIN \
        my_user_films \
    ON \
        u_data.item_id = my_user_films.item_id \
    WHERE \
        my_user_films.rating is null\
    ORDER BY \
        u_data.user_id ")

Уберем всех не значимых пользователей кроме наших ТОП 30

In [28]:
df_pearson_prepare5 = df_pearson_prepare4[df_pearson_prepare4.user_id\
                                          .isin([int(row[0]) for row in pearson_neighbours])]

In [29]:
df_pearson_prepare5.registerTempTable("df_pearson_prepare5")

In [30]:
df_pearson_prepare5.show(5)

+-------+-------+------+--------------+
|user_id|item_id|rating|my_user_rating|
+-------+-------+------+--------------+
|      1|     54|     3|          null|
|      1|     65|     4|          null|
|      1|    270|     5|          null|
|      1|     29|     1|          null|
|      1|     26|     3|          null|
+-------+-------+------+--------------+
only showing top 5 rows



Добавим среднее значение голоса для пользователей

In [31]:
df_pearson_prepare6 = spark.sql("SELECT  \
        df_pearson_prepare5.item_id as item_id,\
        df_pearson_prepare5.user_id as user_id, \
        df_pearson_prepare5.rating as rating,  \
        df_pearson_prepare5.my_user_rating  as my_user_rating, \
        avg_rating_by_user.avg_user_rating as avg_user_rating \
    FROM \
        df_pearson_prepare5 \
    INNER JOIN \
        avg_rating_by_user \
    ON \
        df_pearson_prepare5.user_id = avg_rating_by_user.user_id \
    ORDER BY \
        df_pearson_prepare5.item_id")

In [32]:
pearson_top = df_pearson_prepare6.collect()

In [33]:
len(pearson_top)

3246

In [34]:
from itertools import imap

def predict(s, x):
    # Assume len(x) == len(y)
    num = sum(imap(lambda x, y: x * y, s, x))
    den = sum(imap(lambda x: abs(x), s))
    if den == 0: return 0
    return  num / de

In [35]:
x = []
s = []
predict_top = {}
prev = None
for each in sorted(pearson_top):
    if each[0] != prev and prev != None:
        predict_top[prev]=my_user_average+predict(s, x)
        s = []
        x = []
        s.append(float(pears_correct[each[1]]))
        x.append(float(each[2]-each[4]))
    else:
        s.append(float(pears_correct[each[1]]))
        x.append(float(each[2]-each[4]))
    prev = each[0]

Рекомендуйте пользователю 10 фильмов (pearson_top10), которые он еще не оценивал, с самыми высокими прогнозами из предыдущего пункта.

In [36]:
pearson_top10 = sorted(dict(predict_top).items(), key=lambda x: x[1], reverse=True)[1:11]

In [38]:
[int(row[0]) for row in pearson_top10]

[543, 671, 493, 607, 113, 115, 119, 1211, 635, 653]

# Часть 3. Базовые предикторы:

Глобальное среднее  (average_rating) по всему датасету. Сумма всех оценок по всем фильмам/Количество всех оценок по всем фильмам. (Лабораторная №9 пункт 5 первой части)

In [48]:
t3=t2.map(lambda x: (x[1],x[2]))

In [49]:
global_avg_prepare =t3.map(lambda x: x[1]).aggregate((0,0), \
              lambda acc,value: (acc[0]+value, acc[1]+1),\
              lambda acc1,acc2:(acc1[0]+acc2[0],acc1[1]+acc2[1]))
global_avg_prepare

(352986, 100000)

In [50]:
average_rating = float(global_avg_prepare[0])/float(global_avg_prepare[1])
average_rating

3.52986

Базовый предиктор для каждого пользователя (суммирование по фильмам, оцененным данным пользователем). Здесь Ia - множество фильмов, по которым у пользователя есть рейтинги, а  |Ia| их количество.

In [51]:
bu_1 = t2.map(lambda x: (x[0],x[2])).aggregateByKey((0,0),\
                  lambda acc,value: (acc[0]+value-average_rating, acc[1]+1),\
                  lambda acc1,acc2:(acc1[0]+acc2[0],acc1[1]+acc2[1])).collect()
bu_1

[(2, (11.148679999999999, 62)),
 (4, (19.283360000000002, 24)),
 (6, (22.199540000000066, 211)),
 (8, (15.738260000000015, 59)),
 (10, (124.50576000000012, 184)),
 (12, (43.97714000000003, 51)),
 (14, (55.07372000000005, 98)),
 (16, (111.8196000000001, 140)),
 (18, (97.22878000000013, 277)),
 (20, (-20.43327999999999, 48)),
 (22, (-22.822079999999996, 128)),
 (24, (53.96952000000004, 68)),
 (26, (-62.69501999999996, 107)),
 (28, (15.14106000000002, 79)),
 (30, (10.216020000000006, 43)),
 (32, (-8.724259999999997, 41)),
 (34, (10.402800000000001, 20)),
 (36, (5.402799999999996, 20)),
 (38, (22.88694000000001, 121)),
 (40, (-22.545099999999998, 35)),
 (42, (36.03562000000007, 183)),
 (44, (17.991140000000073, 151)),
 (46, (15.693780000000007, 27)),
 (48, (17.029240000000026, 66)),
 (50, (0.2833599999999965, 24)),
 (52, (41.327840000000016, 56)),
 (54, (10.559100000000026, 65)),
 (56, (20.9161800000001, 187)),
 (58, (47.401560000000075, 154)),
 (60, (115.78912000000014, 208)),
 (62, (-52.

In [57]:
# альтенративаня проверка на DF
#user_rating_count = df.groupBy("user_id").count()
#user_rating_sum = df.groupBy("user_id").sum('rating')
#user_rating_count.join(user_rating_sum, user_rating_count.user_id ==user_rating_sum.user_id ).filter(df.user_id==2).take(5)

In [52]:
bu_2 = map(lambda x: (x[0], x[1][0]/(10+x[1][1])) , bu_1)

In [53]:
bu = dict(bu_2)
bu

{1: 0.07758184397163129,
 2: 0.15484277777777777,
 3: -0.6189443749999997,
 4: 0.5671576470588235,
 5: -0.6201378378378374,
 6: 0.10045040723981931,
 7: 0.42485815980629604,
 8: 0.22809072463768137,
 9: 0.5107212500000002,
 10: 0.6417822680412377,
 11: -0.062328062827225084,
 12: 0.7209367213114759,
 13: -0.42568260061919916,
 14: 0.5099418518518524,
 15: -0.5974161403508769,
 16: 0.7454640000000006,
 17: -0.36410736842105274,
 18: 0.3387762369337984,
 19: 0.013426666666666521,
 20: -0.3522979310344826,
 21: -0.8139943915343916,
 22: -0.1653773913043478,
 23: 0.09932385093167716,
 24: 0.6919169230769235,
 25: 0.46216954545454586,
 26: -0.5358548717948715,
 27: -0.2070428571428573,
 28: 0.17012426966292157,
 29: 0.0905627272727271,
 30: 0.19275509433962273,
 31: 0.30271826086956527,
 32: -0.1710639215686274,
 33: 0.12598117647058807,
 34: 0.34676,
 35: -0.37847142857142857,
 36: 0.1800933333333332,
 37: 0.10146238805970169,
 38: 0.17470946564885503,
 39: 0.1669712499999999,
 40: -0.5010

Базовый предиктор для каждого фильма (суммирование по пользователям, поставившим оценку данному фильму). Здесь Ui - множество пользователей, которые оценили данный фильм, а  |Ui| их количество.

Забродкастим BU

In [54]:
bu_br = sc.broadcast(bu)

In [69]:
bi_1 = t2.map(lambda x: (x[1],x[2]-average_rating-bu_br.value[x[0]])).aggregateByKey((0,0),\
                  lambda acc,value: (acc[0]+value, acc[1]+1),\
                  lambda acc1,acc2:(acc1[0]+acc2[0],acc1[1]+acc2[1])).collect()
bi_1

[(2, (-33.23294458943667, 131)),
 (4, (-4.659366789075319, 209)),
 (6, (2.9347604198133252, 26)),
 (8, (83.98134864440884, 219)),
 (10, (23.717537287854437, 89)),
 (12, (210.50988520065226, 267)),
 (14, (66.78602153591511, 183)),
 (16, (-9.189988163232213, 39)),
 (18, (-5.4203793068695845, 10)),
 (20, (-6.955177190396056, 72)),
 (22, (166.20378329761832, 297)),
 (24, (-11.082109667319926, 174)),
 (26, (-8.263462025853004, 73)),
 (28, (94.11965227855751, 276)),
 (30, (13.604479960717335, 37)),
 (32, (15.709712722237585, 81)),
 (34, (-6.326864721194162, 7)),
 (36, (-16.532119106345483, 13)),
 (38, (-58.7773960478869, 120)),
 (40, (-33.23497036404101, 57)),
 (42, (36.17133265286836, 148)),
 (44, (-16.920050591790904, 79)),
 (46, (4.023313058906477, 27)),
 (48, (56.23992317337525, 117)),
 (50, (445.1260203419549, 583)),
 (52, (19.243953957928476, 91)),
 (54, (-25.155559683020478, 104)),
 (56, (183.23040909161355, 394)),
 (58, (11.028820066022234, 175)),
 (60, (25.5390693164512, 64)),
 (62,

In [62]:
# проверка на DF
#films_rating_count = df.groupBy("item_id").count()
#films_rating_sum = df.groupBy("item_id").sum('rating')
#films_rating_count.join(films_rating_sum, films_rating_sum.item_id ==films_rating_count.item_id ).filter(df.item_id==600).take(5)

In [63]:
#df.filter(df.item_id==600).take(3)

In [64]:
#(6 - bu[7] - bu[932] - 2*average_rating)/(25+2)

Базовый предиктор для каждого фильма (суммирование по пользователям, поставившим оценку данному фильму). Здесь Ui - множество пользователей, которые оценили данный фильм, а  |Ui| их количество.

In [70]:
bi_2 = map(lambda x: (x[0], x[1][0]/(25+x[1][1])) , bi_1)

In [71]:
bi = dict(bi_2)
bi

{1: 0.29292488601631045,
 2: -0.2130316960861325,
 3: -0.32028430592133944,
 4: -0.01991182388493726,
 5: -0.15864301074103926,
 6: 0.05754432195712402,
 7: 0.23308375453915234,
 8: 0.34418585510003624,
 9: 0.29953356633332434,
 10: 0.20804857270047752,
 11: 0.25037393206674036,
 12: 0.7209242643857954,
 13: -0.11385236230753586,
 14: 0.32108664199959186,
 15: 0.1919742326943422,
 16: -0.14359356505050333,
 17: -0.24842958320955694,
 18: -0.15486798019627385,
 19: 0.25604003387822144,
 20: -0.07170285763294904,
 21: -0.5427423674996625,
 22: 0.5161608176944669,
 23: 0.4363346542625048,
 24: -0.055688993303115206,
 25: -0.09720958849843955,
 26: -0.08432104108013269,
 27: -0.2332069339732662,
 28: 0.3126898746795931,
 29: -0.6438257001660421,
 30: 0.21942709614060216,
 31: 0.04916202898192574,
 32: 0.14820483700224138,
 33: -0.03854449234731643,
 34: -0.19771452253731756,
 35: -0.35770498480461305,
 36: -0.43505576595646006,
 37: -0.1890596216901028,
 38: -0.40536135205439244,
 39: -0.1

In [72]:
bi_br = sc.broadcast(bi)

# Часть 4. Item-item CF:

Вычесть из всех рейтингов rui базовый предиктор bui из пункта 4, часть 3 (для всей таблицы рейтингов). Если рейтинга нет, то можно поставить 0.

Фильм, пользователь, Rui-Bui

In [73]:
#t4 = t2.map(lambda x: (x[1], x[0], x[2] - average_rating - bu_br.value[x[0]] - bi_br.value[x[1]] ))
t4 = t2.map(lambda x: (x[1], x[0], x[2] ))
t4.take(5)

[(242, 196, 3), (302, 186, 3), (377, 22, 1), (51, 244, 2), (346, 166, 1)]

In [74]:
t4.filter(lambda x : x[0] == 600).take(5)

[(600, 7, 4), (600, 932, 2)]

In [75]:
#from pyspark.mllib.linalg import Vectors, Matrices

In [76]:
import numpy as np

In [77]:
m = np.zeros(shape=(films_count,users_count))

In [78]:
all_rating = t4.collect()

In [79]:
for item_id, user_id, rating in all_rating:
    m[item_id-1][user_id-1]=rating - average_rating - bu[user_id] - bi[item_id]

In [80]:
#m[600-1][7-1]
#m[600-1][7-1]=4-average_rating - bu[7] - bi[600]
#4-average_rating - bu[7] - bi[600]
#4 - average_rating - bu[7] -bi[600] # Row(user_id=7, item_id=600, rating=4),

In [81]:
for item_id in xrange(0,films_count):
    for user_id in xrange(0,users_count):
        if m[item_id][user_id] == 0:
            m[item_id][user_id] = - average_rating - bu[user_id+1] - bi[item_id+1]

В матрице m строки - все фильмы а столбцы - оценки пользователей для этих фильмов, скоректированные на базовый предиктор

В матрице s будем считать  попарные меры близости (косинус)

In [82]:
s = np.zeros(shape=(films_count,films_count))

In [83]:
s2 = np.zeros(shape=(films_count,films_count))

Найдите попарные меры близости (косинус) для всех фильмов, используя очищенные оценки из пункта 1, часть 4. Суммирование идет по всем пользователям.

In [92]:
from itertools import imap

def cosinus(x, y):
    # Assume len(x) == len(y)
    sum_x_sq = sum(map(lambda x: pow(x, 2), x))
    sum_y_sq = sum(map(lambda x: pow(x, 2), y))
    num = sum(imap(lambda x, y: x * y, x, y))
    den = pow((sum_x_sq ),0.5) * pow((sum_y_sq), 0.5)
    if den == 0: return 0
    return num / den







In [95]:
from tqdm import tqdm

In [96]:
for i in tqdm(xrange(0,films_count)):
    for j in xrange(0,films_count):
        s[i][j]=cosinus(m[i],m[j])

100%|██████████| 1682/1682 [1:45:15<00:00,  4.10s/it]


In [157]:
df_films_to_rating = spark.sql("SELECT  \
        u_data.item_id as item_id\
    FROM \
        u_data \
    LEFT OUTER JOIN \
        my_user_films \
    ON \
        u_data.item_id = my_user_films.item_id \
    WHERE \
        my_user_films.rating is null\
    ORDER BY \
        u_data.user_id ")

In [158]:
films_to_rating =  df_films_to_rating.groupBy("item_id").count().select('item_id').collect()

In [159]:
len(films_to_rating)

1322

In [160]:
# все фильмы оцененные нашим пользователем
my_user_films = df.filter(df.user_id==my_user_id).select('item_id').collect()
my_user_films_set = set({})
for each in my_user_films:
    my_user_films_set.add(each[0])
my_user_films_set

{1,
 3,
 4,
 7,
 8,
 9,
 11,
 12,
 13,
 14,
 15,
 20,
 22,
 23,
 24,
 28,
 32,
 42,
 47,
 48,
 50,
 55,
 56,
 58,
 59,
 60,
 61,
 64,
 69,
 70,
 71,
 79,
 81,
 87,
 89,
 92,
 93,
 95,
 96,
 97,
 98,
 99,
 100,
 109,
 116,
 117,
 118,
 121,
 122,
 123,
 124,
 125,
 127,
 129,
 132,
 134,
 135,
 137,
 140,
 144,
 147,
 148,
 149,
 150,
 151,
 157,
 168,
 169,
 170,
 172,
 173,
 174,
 176,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 187,
 188,
 189,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 201,
 202,
 203,
 204,
 215,
 216,
 221,
 222,
 223,
 224,
 234,
 235,
 236,
 237,
 238,
 242,
 243,
 245,
 246,
 248,
 249,
 250,
 251,
 252,
 253,
 255,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 268,
 269,
 271,
 272,
 273,
 276,
 281,
 282,
 283,
 285,
 286,
 287,
 288,
 289,
 291,
 292,
 293,
 294,
 295,
 297,
 298,
 299,
 301,
 302,
 303,
 305,
 306,
 307,
 312,
 313,
 315,
 318,
 319,
 320,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 330,
 331,
 332,
 333,
 334,
 336

In [161]:
predicators_top = {}
for each in tqdm(films_to_rating):
    film = each[0]
    #топ 30 соседей для фильма
    vector = zip(xrange(0,films_count),s[film-1])
    top10 = [x+1 for x,y in sorted(dict(vector).items(), key=lambda x: x[1], reverse=True)[1:11]] 
    #предскажем рейтинг фильма
    result =  average_rating + bu[my_user_id] + bi[film] + predict_film(top10, film)
    predicators_top[film]=result

100%|██████████| 1322/1322 [00:02<00:00, 620.07it/s]


In [164]:
predicators_top10 = sorted(dict(predicators_top).items(), key=lambda x: float(x[1]), reverse=True)[:10] 

In [165]:
len(predicators_top10)

10

In [166]:
print [x for x,y in predicators_top10]

[651, 156, 199, 523, 496, 209, 385, 161, 175, 210]


In [167]:
# все оценки нашего пользователя
my_user_ratings = df.filter(df.user_id==my_user_id).select("item_id", 'rating').collect()

In [168]:
my_user_ratings_dict = dict(my_user_ratings)

In [169]:
def predict_film(top30_vec, film_id):
    den = 0
    num = 0
    for each in top30_vec:
        if set({each}).issubset(my_user_films_set):
            num = num + s[each-1][film_id-1]*(my_user_ratings_dict[each] -  average_rating - bu[my_user_id] - bi[each]) 
            den = den + abs(s[each-1][film_id-1]) 
    if den == 0: return 0
    return num / den

Формируем json с результатами

In [146]:
print(json.dumps({'average_film_ratings':round(average_film_ratings,4), \
                  'average_rating': round(average_rating,4), \
                  'average_user_ratings': round(average_user_ratings,4), \
                  'completeness': round(completeness, 4), \
                  'pearson_neighbours': [x for x,y in pearson_neighbours], \
                  'pearson_top10': [x for x,y in pearson_top10],\
                  'predicators_positive_top10': [x for x,y in predicators_top10],\
                  'predicators_top10': [x for x,y in predicators_top10]\
                 }, sort_keys=True))

{"average_film_ratings": 59.453, "average_rating": 3.5299, "average_user_ratings": 106.0445, "completeness": 0.063, "pearson_neighbours": [22, 303, 294, 276, 267, 117, 757, 453, 457, 177, 640, 560, 293, 851, 622, 592, 70, 58, 868, 397, 5, 92, 381, 892, 435, 650, 661, 389, 758, 890], "pearson_top10": [513, 45, 285, 610, 736, 835, 965, 1451, 611, 1143], "predicators_positive_top10": [520, 136, 197, 659, 603, 510, 193, 663, 427, 514], "predicators_top10": [520, 136, 197, 659, 603, 510, 193, 663, 427, 514]}


In [170]:
print(json.dumps({'average_rating': round(average_rating,4), \
                  'predicators_positive_top10': [x for x,y in predicators_top10],\
                  'predicators_top10': [x for x,y in predicators_top10]\
                 }, sort_keys=True))

{"average_rating": 3.5299, "predicators_positive_top10": [651, 156, 199, 523, 496, 209, 385, 161, 175, 210], "predicators_top10": [651, 156, 199, 523, 496, 209, 385, 161, 175, 210]}
